<a href="https://colab.research.google.com/github/milesba4/CS158-ML/blob/main/homework11%20(Softmax%20SGD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Homework 11**

We'll start with a few imports and our running library of functions.

In [ ]:
import pandas as pd
import math
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.datasets import load_digits


def TrainTestSplit(X,y,p,seed=1):
  '''Splits feature matrix X and target array y into train and test sets
  p is the fraction going to train'''
  np.random.seed(seed) #controls randomness
  size=len(y) 
  train_size=int(p*size)
  train_mask=np.zeros(size,dtype=bool)
  train_indices=np.random.choice(size, train_size, replace=False)
  train_mask[train_indices]=True 
  test_mask=~train_mask
  X_train=X[train_mask]
  X_test=X[test_mask]
  y_train=y[train_mask]
  y_test=y[test_mask]
  return X_train,X_test,y_train,y_test

def PolyFeatures(x,d):
  X=np.zeros((len(x),d+1))
  for i in range(d+1):
    X[:,i]=x**i
  return X

def AddOnes(X):
  return np.concatenate((X,np.ones((len(X),1))),axis=1)

class Scaler:
  def __init__(self,z):
    self.min=np.min(z,axis=0)
    self.max=np.max(z,axis=0)

  def scale(self,x):
    
    return (x-self.min)/(self.max-self.min+0.000001)

  def unscale(self,x):
    return x*(self.max-self.min+0.000001)+self.min


def LinearSGD(X,y,epochs,batch_size,lr,alpha=0,beta=0):
  '''Stochastic Gradient Descent With L1 and L2 regularization'''
  #alpha=amount of L1 (Lasso) regularization
  #beta=amount of L2 (Ridge) regularization
  X=np.array(X) #Just in case X is a DataFrame
  y=np.array(y) #Just in case y is a Series
  n=len(X)
  coeff=np.ones(X.shape[1]) #Initialize all coeff to be 1 (something to play with?)
  indices=np.arange(len(X))
  for i in range(epochs):
    np.random.seed(i)
    np.random.shuffle(indices)
    X_shuffle=X[indices] 
    y_shuffle=y[indices] 
    num_batches=n//batch_size
    for j in range(num_batches):
      X_batch=X_shuffle[j*batch_size:(j+1)*batch_size]
      y_batch=y_shuffle[j*batch_size:(j+1)*batch_size]
      resid=X_batch@coeff-y_batch
      gradient=lr*((X_batch.T)@resid)/len(X_batch)+alpha*(2*(coeff>0)-1)+beta*coeff
      coeff=coeff-gradient #Gradient Descent step.
    if n%batch_size!=0: #Check if there is a smaller leftover batch
      X_batch=X_shuffle[num_batches*batch_size:] #last batch
      y_batch=y_shuffle[num_batches*batch_size:] #last batch
      resid=X_batch@coeff-y_batch
      gradient=lr*((X_batch.T)@resid)/len(X_batch)+alpha*(2*(coeff>0)-1)+beta*coeff
      coeff=coeff-gradient 
  return coeff

def LinearPredict(X,coeff): #If X was scaled, then this will return scaled predictions
  return X@coeff

def MSE(pred,y):
  return np.sum((pred-y)**2)/len(y)

def sigmoid(t):
  return 1/(1+np.exp(-t))

def LogisticProbability(X,coeff):
  return sigmoid(X@coeff)

def LogisticPredict(X,coeff):
  return X@coeff>0

def accuracy(pred,y):
  return np.sum(pred==y)/len(y)

def LogLoss(probs,y):
  return np.sum(y*np.log(probs)+(1-y)*np.log(1-probs))/len(y)

def LogisticSGD(X,y,epochs,batch_size,lr,alpha=0,beta=0):
  '''Stochastic Gradient Descent for Logistic Regression'''
  #alpha=amount of L1 (Lasso) regularization
  #beta=amount of L2 (Ridge) regularization
  X=np.array(X) 
  y=np.array(y) 
  n=len(X)
  coeff=np.ones(X.shape[1]) 
  indices=np.arange(len(X))
  for i in range(epochs):
    np.random.seed(i)
    np.random.shuffle(indices)
    X_shuffle=X[indices] 
    y_shuffle=y[indices] 
    num_batches=n//batch_size
    for j in range(num_batches):
      X_batch=X_shuffle[j*batch_size:(j+1)*batch_size]
      y_batch=y_shuffle[j*batch_size:(j+1)*batch_size]
      probs=LogisticProbability(X_batch,coeff)
      grad=X_batch.T@(probs-y_batch)/len(X_batch)
      gradient=lr*grad+alpha*(2*(coeff>0)-1)+beta*coeff
      coeff=coeff-gradient 
    if n%batch_size!=0: #Check if there is a smaller leftover batch
      X_batch=X_shuffle[num_batches*batch_size:] #last batch
      y_batch=y_shuffle[num_batches*batch_size:] #last batch
      probs=LogisticProbability(X_batch,coeff)
      grad=X_batch.T@(probs-y_batch)/len(X_batch)
      gradient=lr*grad+alpha*(2*(coeff>0)-1)+beta*coeff
      coeff=coeff-gradient
  return coeff


Write a function that takes a feature matrix `X` with shape (m,n), and a coefficient matrix `coeff` of shape (n,k), and returns an array with m elements that gives the predicted class of a softmax regression model. Here, m is the number of observations (rows of `X`), n is the number of features (columns of `X`), and k is the number of classes.  The `i`th element of the array that is returned should be the column number of the maximum element of the `i`th row of `X` times `coeff`.

In [ ]:
def SoftmaxPredict(X,coeff):
  return np.argmax(X@coeff, axis=1)

Write a function that takes a feature matrix `X` and a coefficient matrix `coeff` (as in the previous problem), and returns a matrix of predicted probabilities of shape (m,k). The entry in row `i`, column `j`, is the probability that the observation in row `i` of X is in class `j`. This is found by the formula
$$\frac{exp(s^j_i)}{\sum \limits _{j=1} ^k exp(s^j_i)},$$
where $s^j_i$ is the entry in row `i`, column `j`, of the matrix that is the product of `X` and `coeff`. Note that this formula will always be a number between 0 and 1.

In [ ]:
def SoftmaxProbability(X,coeff): 
  #print("X", X)
  return (math.e**(X@coeff))/np.sum(math.e**(X@coeff), axis = 1)[:,np.newaxis]

Write a function `OneHot` that takes a target array `y` with m entries, where each entry is one of k possible classes, and returns a matrix of shape (m,k), where each entry is a 1 or a 0. If there is a 1 in row `i`, column `j`,  of OneHot(y), that would indicate that the `i`th entry of `y` is the number `j`. 

In [ ]:
def OneHot(y):
  # pass y into get dummies
  return np.array(pd.get_dummies(y))
  #turn rersult into np.array



The function `SoftmaxSGD` finds the coefficients of softmax regression by gradient descent. The code is almost identical to the LogisticSGD function at the top of this colab. Read the code below, paying special attention ot the comments, and complete the two missing lines indicated by "#YOUR CODE HERE". 

Note that the gradient of the categorical cross-entropy function (the loss function used in Softmax regression) is given by 
$$X^{T}(probs-\overline{y})$$
where $probs$ is given by `SoftmaxProbability(X,coeff)` and $\overline{y}$ is the One Hot enoding of the target vector $y$. 

In [ ]:
def SoftmaxSGD(X,y,epochs,batch_size,lr,alpha=0,beta=0):
  '''Stochastic Gradient Descent for Softmax Regression'''
  X=np.array(X) 
  y=np.array(y)
  classes=np.max(y)+1 #THIS IS NEW!
  ybar=OneHot(y) #THIS IS NEW!
  n=len(X)
  coeff=np.ones((X.shape[1],classes)) #initial coeff matrix. Note the shape!
  indices=np.arange(len(X))
  for i in range(epochs):
    np.random.seed(i)
    np.random.shuffle(indices)
    X_shuffle=X[indices] 
    y_shuffle=ybar[indices] #NOTE THE CHANGE FROM y to ybar
    num_batches=n//batch_size
    for j in range(num_batches):
      X_batch=X_shuffle[j*batch_size:(j+1)*batch_size]
      y_batch=y_shuffle[j*batch_size:(j+1)*batch_size]
      probs=SoftmaxProbability(X_batch,coeff)
      grad=X_batch.T@(probs-y_batch)/len(X_batch) #NOTHING NEW HERE!!!!
      gradient=lr*grad+alpha*(2*(coeff>0)-1)+beta*coeff
      coeff=coeff-gradient 
    if n%batch_size!=0: #Check if there is a smaller leftover batch
      X_batch=X_shuffle[num_batches*batch_size:] 
      y_batch=y_shuffle[num_batches*batch_size:] 
      probs=SoftmaxProbability(X_batch,coeff)
      grad=X_batch.T@(probs-y_batch)/len(X_batch)
      gradient=lr*grad+alpha*(2*(coeff>0)-1)+beta*coeff
      coeff=coeff-gradient
  return coeff


We now load the `digits` feature matrix and target that we first encountered in Homework 4. 

In [ ]:
X=load_digits().data
y=load_digits().target


As usual, do the following:
1. 80/20 Test-train split to obtain Xtrain, Xtest, ytrain and ytest
2. Scale Xtrain and Xtest
3. Add a column of ones to Xtrain and Xtest

In [ ]:
Xtrain,Xtest,ytrain,ytest = TrainTestSplit(X,y,.8,seed=1)
#Scale training feature matrix and test feature matrix
Xtrain_scaler = Scaler(Xtrain)

Xtrain_scaled = Xtrain_scaler.scale(Xtrain)
Xtest_scaled = Xtrain_scaler.scale(Xtest)



# Add a column of ones to Xtrain and Xtest
Xtrain_scaled=AddOnes(Xtrain_scaled)
Xtest_scaled=AddOnes(Xtest_scaled)


Find the coefficient matrix of a softmax regression model to predict which digit is which. Use 10000 epochs, with batch sizes of 5000, and a learning rate of 0.01.

In [ ]:
# (X,y,epochs,batch_size,lr,alpha=0,beta=0)
coeff=SoftmaxSGD(Xtrain_scaled,ytrain,10000,5000,.01,alpha=0, beta=0)
coeff

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 0.94443274,  0.88071394,  1.14591728,  1.13947983,  0.95433782,
         1.21867171,  0.91585354,  0.95108033,  0.91200204,  0.93751078],
       [ 0.88310184,  0.65151415,  1.45632003,  1.33541372,  0.488992  ,
         1.8610445 ,  0.42250768,  1.06201593,  0.79297247,  1.04611768],
       [ 1.1867326 ,  0.68173831,  1.23112453,  1.37335271,  0.33783946,
         1.13979016,  0.85539953,  1.3072843 ,  0.72842438,  1.15831403],
       [ 0.89611943,  0.29534135,  0.91743008,  1.65094229,  0.90073255,
         1.36967183,  0.81125922,  1.33688965,  1.00208769,  0.81952591],
       [ 0.5744144 ,  1.13533929,  0.6966702 ,  1.42808316,  0.27429456,
         2.0717397 ,  0.55238845,  1.55329518,  0.91721456,  0.7965605 ],
       [ 0.8090978 ,  0.93933481,  0.85243399,  0.95439355,  0.73079375,
         1.48742226,  0.87494628,  1.58622007

Generate predictions for your model on the test set Xtest.

In [ ]:
pred=SoftmaxPredict(Xtest_scaled,coeff)
pred


array([5, 0, 1, 4, 5, 7, 7, 0, 3, 1, 0, 7, 8, 4, 5, 1, 1, 0, 8, 1, 3, 4,
       5, 9, 0, 2, 5, 0, 1, 4, 7, 5, 7, 8, 6, 9, 8, 0, 1, 3, 7, 0, 1, 7,
       2, 0, 8, 7, 0, 3, 2, 3, 6, 2, 3, 7, 3, 4, 5, 6, 0, 2, 5, 9, 5, 2,
       9, 4, 7, 0, 9, 6, 9, 3, 7, 2, 6, 4, 6, 1, 0, 2, 0, 6, 3, 2, 3, 3,
       1, 3, 4, 9, 8, 8, 8, 8, 0, 6, 2, 6, 9, 5, 0, 2, 8, 4, 6, 1, 2, 0,
       1, 1, 1, 1, 0, 9, 7, 4, 4, 8, 5, 4, 0, 2, 0, 1, 0, 2, 5, 4, 7, 7,
       0, 2, 3, 6, 4, 9, 5, 8, 7, 6, 7, 1, 7, 3, 9, 1, 8, 6, 2, 5, 0, 5,
       8, 2, 7, 6, 5, 9, 3, 1, 6, 5, 3, 6, 1, 7, 2, 5, 9, 1, 2, 5, 6, 9,
       3, 8, 7, 7, 6, 3, 3, 6, 5, 0, 9, 0, 3, 4, 0, 7, 0, 2, 5, 0, 7, 0,
       1, 3, 7, 5, 8, 7, 5, 0, 7, 9, 9, 0, 3, 2, 1, 4, 8, 4, 9, 3, 4, 7,
       8, 0, 5, 6, 0, 3, 0, 2, 3, 6, 2, 0, 0, 6, 4, 3, 0, 7, 5, 4, 8, 9,
       7, 8, 2, 3, 6, 8, 9, 4, 5, 5, 6, 9, 9, 8, 3, 2, 7, 6, 3, 3, 3, 6,
       9, 9, 2, 0, 1, 1, 6, 6, 5, 7, 4, 8, 2, 0, 6, 8, 9, 7, 5, 8, 3, 9,
       5, 2, 2, 0, 9, 3, 4, 9, 0, 6, 7, 7, 5, 8, 8,

Evaluate the accuracy of those predictions. (You may use the `accuracy` function at the top of this colab.)

In [ ]:
acc=accuracy(pred,ytest)
acc

0.9444444444444444